# Procesamiento de datos

## Procedimientos

In [1]:
import pandas as pd

In [2]:
procedimientos = pd.read_csv('procedimientos.csv')

# Seleccionar solo los procedimientos del Banco Central
procedimientos = procedimientos[procedimientos['cedula_institucion'] == 4000004017]

In [3]:
procedimientos.drop(['fecha_limite_recursos', 'pago_adelantado_pymes', 'fecha_invitacion', 'nombre_unidad_compra', 'excepcion_cd'], axis=1, inplace=True)
procedimientos.dropna(subset=['fecha_inicio_recepcion', 'fecha_cierre_recepcion', 'fecha_apertura'], inplace=True)

procedimientos = pd.DataFrame(procedimientos)
vacio = procedimientos.isnull().sum()
vacio

nro_sicop                  0
cedula_institucion         0
fecha_publicacion          0
fecha_inicio_recepcion     0
fecha_cierre_recepcion     0
fecha_apertura             0
nro_procedimiento          0
cod_unidad_compra          0
tipo_procedimiento         0
modalidad_procedimiento    0
descripcion                0
dtype: int64

## Lineas procedimientos

In [4]:
lineas_proc = pd.read_csv('lineasproc.csv')
lineas_proc = pd.DataFrame(lineas_proc)

In [5]:
data = procedimientos.merge(lineas_proc, on='nro_sicop', how='left')
data

,nro_sicop,cedula_institucion_x,fecha_publicacion_x,fecha_inicio_recepcion_x,fecha_cierre_recepcion_x,fecha_apertura_x,nro_procedimiento_x,cod_unidad_compra_x,tipo_procedimiento_x,modalidad_procedimiento_x,...,tipo_procedimiento_y,modalidad_procedimiento_y,descripcion_y,numero_linea,numero_partida,cantidad_solicitada,precio_unitario_estimado,tipo_cambio_dolar,codigo_identificacion,total
0,20160800993,4000004017,2016-08-26 10:15:13.000,2016-08-26 16:00:00.000,2016-09-26 10:00:00.000,2016-09-26 10:00:00.000,2016LA-000027-0004900001,1,LICITACIÓN ABREVIADA,Servicios,...,LICITACIÃ“N ABREVIADA,Servicios,DESARROLLO SITIO WEB BCCR,1,1,1,60000000.0,558.69,8.111210e+15,60000000.0
1,20160700910,4000004017,2016-07-26 10:30:49.000,2016-07-26 11:00:00.000,2016-08-03 11:00:00.000,2016-08-03 11:00:00.000,2016CD-000074-0004900001,1,CONTRATACIÓN DIRECTA,Cantidad definida,...,CONTRATACIÃ“N DIRECTA,Cantidad definida,CompraÂ Directa COMPRAÂ DEÂ MÃQUINAÂ DEÂ SUMA...,1,1,2,100000.0,554.30,4.410180e+15,200000.0
2,20160701174,4000004017,2016-08-10 15:32:14.000,2016-08-10 15:40:00.000,2016-08-18 15:00:00.000,2016-08-18 15:00:00.000,2016CD-000083-0004900001,1,CONTRATACIÓN DIRECTA,Según demanda,...,CONTRATACIÃ“N DIRECTA,SegÃºn demanda,CONTRATACIÃ“NÂ DEÂ UNAÂ EMPRESAÂ QUEÂ BRINDEÂ ...,2,1,10,26500.0,556.33,1.411170e+15,265000.0
3,20160701174,4000004017,2016-08-10 15:32:14.000,2016-08-10 15:40:00.000,2016-08-18 15:00:00.000,2016-08-18 15:00:00.000,2016CD-000083-0004900001,1,CONTRATACIÓN DIRECTA,Según demanda,...,CONTRATACIÃ“N DIRECTA,SegÃºn demanda,CONTRATACIÃ“NÂ DEÂ UNAÂ EMPRESAÂ QUEÂ BRINDEÂ ...,1,1,80,32000.0,556.33,1.411170e+15,2560000.0
4,20160800177,4000004017,2016-08-08 15:05:15.000,2016-08-08 15:30:00.000,2016-08-16 15:00:00.000,2016-08-16 15:00:00.000,2016CD-000080-0004900001,1,CONTRATACIÓN DIRECTA,Cantidad definida,...,CONTRATACIÃ“N DIRECTA,Cantidad definida,RenovaciÃ³nÂ deÂ 35Â licenciasÂ deÂ AntivirusÂ...,1,1,1,919564.8,556.03,8.111220e+15,919564.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7871,20230700350,4000004017,2023-07-07 08:25:35.000,2023-07-07 08:30:00.000,2023-07-21 10:00:00.000,2023-07-21 10:00:00.000,2023LE-000014-0004900001,1,LICITACIÓN MENOR,Servicios,...,LICITACIÃ“N MENOR,Servicios,ContrataciÃ³nÂ deÂ unaÂ firmaÂ auditoraÂ exter...,1,1,1,94976500.0,553.52,8.411170e+15,94976500.0
7872,20230700093,4000004017,2023-07-03 16:15:20.000,2023-07-03 16:30:00.000,2023-07-07 14:00:00.000,2023-07-07 14:00:00.000,2023PX-000179-0004900001,1,PROCEDIMIENTO POR EXCEPCIÓN,Servicios,...,PROCEDIMIENTO POR EXCEPCIÃ“N,Servicios,InscripciÃ³nÂ deÂ 8 funcionarios de SUPEN en R...,1,1,1,7983954.0,549.50,8.610180e+15,7983954.0
7873,20230601248,4000004017,2023-06-09 17:05:55.000,2023-06-12 10:00:00.000,2023-06-13 10:00:00.000,2023-06-13 10:00:00.000,2023PX-000149-0004900001,1,PROCEDIMIENTO POR EXCEPCIÓN,Servicios,...,PROCEDIMIENTO POR EXCEPCIÃ“N,Servicios,InscripciÃ³n en actividad de capacitaciÃ³n La ...,1,1,1,275400.0,546.19,8.610180e+15,275400.0
7874,20230400095,4000004017,2023-04-05 16:03:03.000,2023-04-05 16:03:00.000,2023-04-13 10:00:00.000,2023-04-13 10:00:00.000,2023PX-000088-0004900001,1,PROCEDIMIENTO POR EXCEPCIÓN,Cantidad definida,...,PROCEDIMIENTO POR EXCEPCIÃ“N,Cantidad definida,SEÂ REQUIEREÂ INSCRIBIRÂ 1Â FUNCIONARIOÂ DEÂ L...,1,1,1,65000.0,543.31,8.610180e+15,65000.0


In [6]:
duplicados_nro_sicop = data[data.duplicated('nro_sicop', keep=False)]

# Cuenta cuántas veces se repite cada 'nro_sicop' duplicado
duplicados_cuenta = duplicados_nro_sicop['nro_sicop'].value_counts()

duplicados_cuenta_df = duplicados_cuenta.reset_index()
duplicados_cuenta_df.columns = ['nro_sicop', 'count']

# Función para calcular el total_procedimiento para un nro_sicop dado
def calcular_total_procedimiento(nro_sicop):
    n = data[data['nro_sicop'] == nro_sicop]
    total = n.sort_values(by='numero_linea')
    total_procedimiento = total['total'].sum()
    return total_procedimiento

# Aplicar la función a cada nro_sicop en duplicados_cuenta_df
duplicados_cuenta_df['total_procedimiento'] = duplicados_cuenta_df['nro_sicop'].apply(calcular_total_procedimiento)

# Establecer el formato de visualización de los números flotantes
pd.options.display.float_format = '{:,.2f}'.format

# Mostrar el DataFrame resultante
print(duplicados_cuenta_df)

       nro_sicop  count  total_procedimiento
0    20190801073    260         1,245,485.58
1    20191102039    155           365,573.01
2    20180400710    120        37,555,763.00
3    20151100765     79       127,148,140.30
4    20210900913     79         2,797,625.52
..           ...    ...                  ...
778  20190200443      2         4,932,152.95
779  20211100366      2         1,850,725.50
780  20211101174      2        10,000,000.00
781  20211002960      2         5,390,000.00
782  20160701174      2         2,825,000.00

[783 rows x 3 columns]


In [7]:
# Filtrar los valores no duplicados en 'nro_sicop'
no_duplicados_nro_sicop = data[~data.duplicated('nro_sicop', keep=False)]

# Crear un DataFrame con la columna 'nro_sicop' y la suma de 'precio_unitario_estimado' como 'total'
no_duplicados_total = no_duplicados_nro_sicop.groupby('nro_sicop')['total'].sum().reset_index()

# Renombrar la columna 'precio_unitario_estimado' a 'total'
no_duplicados_total.columns = ['nro_sicop', 'total_procedimiento']

In [8]:
# Suponiendo que 'duplicated_counts_df' ya tiene la columna 'total_procedimiento'
duplicados_cuenta_df = duplicados_cuenta_df[['nro_sicop', 'total_procedimiento']]

# Renombrar la columna 'total' a 'total_procedimiento' en non_duplicated_totals
no_duplicados_total.columns = ['nro_sicop', 'total_procedimiento']

# Concatenar los dos DataFrames
final_procedimientos = pd.concat([duplicados_cuenta_df, no_duplicados_total], ignore_index=True)

## Adjudicaciones

In [9]:
data = pd.read_csv('adjudicaciones.csv')
data.drop(['desierto', 'es_valido', 'version', 'fecha_comunicacion', 'permite_recursos'], axis=1, inplace=True)

# Verificar y eliminar duplicados en la tabla data
data.drop_duplicates(subset=['nro_sicop'], inplace=True)

# Cargar los procedimientos y filtrar por cedula_institucion
procedimientos = pd.read_csv('procedimientos.csv')
procedimientos = procedimientos[procedimientos['cedula_institucion'] == 4000004017]

# Verificar y eliminar duplicados en la tabla procedimientos
procedimientos.drop_duplicates(subset=['nro_sicop'], inplace=True)

# Unir las tablas por el campo nro_sicop
combinada_data = pd.merge(data, procedimientos, on='nro_sicop', how='inner')

# Eliminar las columnas innecesarias
combinada_data.drop(['fecha_limite_recursos', 'pago_adelantado_pymes', 'fecha_invitacion', 'nombre_unidad_compra', 'excepcion_cd'], axis=1, inplace=True)

# Eliminar filas con valores nulos en las columnas especificadas
combinada_data.dropna(subset=['fecha_inicio_recepcion', 'fecha_cierre_recepcion', 'fecha_apertura'], inplace=True)

# Verificar y eliminar duplicados en la tabla fusionada
combinada_data.drop_duplicates(subset=['nro_sicop'], inplace=True)

# Verificar la tabla resultante
combinada_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3715 entries, 0 to 3727
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   nro_sicop                3715 non-null   int64 
 1   nro_acto                 3715 non-null   int64 
 2   fecha_adj_firme          3715 non-null   object
 3   cedula_institucion       3715 non-null   int64 
 4   fecha_publicacion        3715 non-null   object
 5   fecha_inicio_recepcion   3715 non-null   object
 6   fecha_cierre_recepcion   3715 non-null   object
 7   fecha_apertura           3715 non-null   object
 8   nro_procedimiento        3715 non-null   object
 9   cod_unidad_compra        3715 non-null   int64 
 10  tipo_procedimiento       3715 non-null   object
 11  modalidad_procedimiento  3715 non-null   object
 12  descripcion              3715 non-null   object
dtypes: int64(4), object(9)
memory usage: 406.3+ KB


In [10]:
combinada_data.head()

,nro_sicop,nro_acto,fecha_adj_firme,cedula_institucion,fecha_publicacion,fecha_inicio_recepcion,fecha_cierre_recepcion,fecha_apertura,nro_procedimiento,cod_unidad_compra,tipo_procedimiento,modalidad_procedimiento,descripcion
0,20230200428,833534,2023-02-16 18:16:18.000,4000004017,2023-02-10 09:17:28.000,2023-02-10 09:20:00.000,2023-02-15 10:00:00.000,2023-02-15 10:00:00.000,2023PX-000033-0004900001,1,PROCEDIMIENTO POR EXCEPCIÓN,Cantidad definida,SE REQUIERE INSCRIBIR 2 FUNCIONARIOS DE LA DIV...
1,20230300247,836311,2023-03-06 12:51:40.000,4000004017,2023-03-02 17:46:47.000,2023-03-02 17:48:00.000,2023-03-06 10:00:00.000,2023-03-06 10:00:00.000,2023PX-000054-0004900001,1,PROCEDIMIENTO POR EXCEPCIÓN,Servicios,Inscripción a 5 funcionarios de la Sugese:1. A...
2,20230201566,835891,2023-03-03 08:34:17.000,4000004017,2023-02-27 16:09:16.000,2023-02-27 16:15:00.000,2023-03-02 10:00:00.000,2023-03-02 10:00:00.000,2023PX-000050-0004900001,1,PROCEDIMIENTO POR EXCEPCIÓN,Cantidad definida,SE REQUIERE INSCRIBIR 1 FUNCIONARIA DE LA DIVI...
3,20230301392,840622,2023-03-27 13:32:29.000,4000004017,2023-03-17 10:06:09.000,2023-03-17 10:30:00.000,2023-03-24 10:30:00.000,2023-03-24 10:30:00.000,2023PX-000064-0004900001,1,PROCEDIMIENTO POR EXCEPCIÓN,Servicios,Inscripción curso: Análisis de datos en Micros...
4,20230302152,841318,2023-03-28 22:00:32.000,4000004017,2023-03-24 08:12:06.000,2023-03-24 09:00:00.000,2023-03-28 09:00:00.000,2023-03-28 09:00:00.000,2023PX-000074-0004900001,1,PROCEDIMIENTO POR EXCEPCIÓN,Servicios,"Inscripción: El compromiso en el liderazgo, un..."


In [11]:
# DF con el campo calculado del gasto por procedimiento
final_df = combinada_data.merge(final_procedimientos, on='nro_sicop', how='left')
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3715 entries, 0 to 3714
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   nro_sicop                3715 non-null   int64  
 1   nro_acto                 3715 non-null   int64  
 2   fecha_adj_firme          3715 non-null   object 
 3   cedula_institucion       3715 non-null   int64  
 4   fecha_publicacion        3715 non-null   object 
 5   fecha_inicio_recepcion   3715 non-null   object 
 6   fecha_cierre_recepcion   3715 non-null   object 
 7   fecha_apertura           3715 non-null   object 
 8   nro_procedimiento        3715 non-null   object 
 9   cod_unidad_compra        3715 non-null   int64  
 10  tipo_procedimiento       3715 non-null   object 
 11  modalidad_procedimiento  3715 non-null   object 
 12  descripcion              3715 non-null   object 
 13  total_procedimiento      3715 non-null   float64
dtypes: float64(1), int64(4),

In [12]:
# Contar los valores NA en cada columna
na_counts = final_df.isna().sum()

# Mostrar los resultados
print(na_counts)

nro_sicop                  0
nro_acto                   0
fecha_adj_firme            0
cedula_institucion         0
fecha_publicacion          0
fecha_inicio_recepcion     0
fecha_cierre_recepcion     0
fecha_apertura             0
nro_procedimiento          0
cod_unidad_compra          0
tipo_procedimiento         0
modalidad_procedimiento    0
descripcion                0
total_procedimiento        0
dtype: int64


In [13]:
final_df.drop(['cod_unidad_compra'], axis=1, inplace=True)

In [14]:
# Convertir las columnas de fecha a datetime
final_df['fecha_cierre_recepcion'] = pd.to_datetime(final_df['fecha_cierre_recepcion'])
final_df['fecha_adj_firme'] = pd.to_datetime(final_df['fecha_adj_firme'])

# Calcular la diferencia en días y crear la nueva columna 'duracion'
final_df['duracion'] = (final_df['fecha_adj_firme'] - final_df['fecha_cierre_recepcion']).dt.days

# Mostrar las primeras filas para verificar
final_df.head()


,nro_sicop,nro_acto,fecha_adj_firme,cedula_institucion,fecha_publicacion,fecha_inicio_recepcion,fecha_cierre_recepcion,fecha_apertura,nro_procedimiento,tipo_procedimiento,modalidad_procedimiento,descripcion,total_procedimiento,duracion
0,20230200428,833534,2023-02-16 18:16:18,4000004017,2023-02-10 09:17:28.000,2023-02-10 09:20:00.000,2023-02-15 10:00:00,2023-02-15 10:00:00.000,2023PX-000033-0004900001,PROCEDIMIENTO POR EXCEPCIÓN,Cantidad definida,SE REQUIERE INSCRIBIR 2 FUNCIONARIOS DE LA DIV...,"218,280.00",1
1,20230300247,836311,2023-03-06 12:51:40,4000004017,2023-03-02 17:46:47.000,2023-03-02 17:48:00.000,2023-03-06 10:00:00,2023-03-06 10:00:00.000,2023PX-000054-0004900001,PROCEDIMIENTO POR EXCEPCIÓN,Servicios,Inscripción a 5 funcionarios de la Sugese:1. A...,"765,000.00",0
2,20230201566,835891,2023-03-03 08:34:17,4000004017,2023-02-27 16:09:16.000,2023-02-27 16:15:00.000,2023-03-02 10:00:00,2023-03-02 10:00:00.000,2023PX-000050-0004900001,PROCEDIMIENTO POR EXCEPCIÓN,Cantidad definida,SE REQUIERE INSCRIBIR 1 FUNCIONARIA DE LA DIVI...,"91,800.00",0
3,20230301392,840622,2023-03-27 13:32:29,4000004017,2023-03-17 10:06:09.000,2023-03-17 10:30:00.000,2023-03-24 10:30:00,2023-03-24 10:30:00.000,2023PX-000064-0004900001,PROCEDIMIENTO POR EXCEPCIÓN,Servicios,Inscripción curso: Análisis de datos en Micros...,"408,000.00",3
4,20230302152,841318,2023-03-28 22:00:32,4000004017,2023-03-24 08:12:06.000,2023-03-24 09:00:00.000,2023-03-28 09:00:00,2023-03-28 09:00:00.000,2023PX-000074-0004900001,PROCEDIMIENTO POR EXCEPCIÓN,Servicios,"Inscripción: El compromiso en el liderazgo, un...","6,426,875.00",0


In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3715 entries, 0 to 3714
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   nro_sicop                3715 non-null   int64         
 1   nro_acto                 3715 non-null   int64         
 2   fecha_adj_firme          3715 non-null   datetime64[ns]
 3   cedula_institucion       3715 non-null   int64         
 4   fecha_publicacion        3715 non-null   object        
 5   fecha_inicio_recepcion   3715 non-null   object        
 6   fecha_cierre_recepcion   3715 non-null   datetime64[ns]
 7   fecha_apertura           3715 non-null   object        
 8   nro_procedimiento        3715 non-null   object        
 9   tipo_procedimiento       3715 non-null   object        
 10  modalidad_procedimiento  3715 non-null   object        
 11  descripcion              3715 non-null   object        
 12  total_procedimiento      3715 non-

In [16]:
nombre_archivo = 'df_final.csv'
final_df.to_csv(nombre_archivo, index=False)

# Verificación
print(f"El DataFrame ha sido exportado a {nombre_archivo}")

El DataFrame ha sido exportado a df_final.csv


In [17]:
df = pd.read_csv('df_final.csv')

In [18]:
df['fecha_adj_firme'] = pd.to_datetime(df['fecha_adj_firme'])

# Crear una columna para el período (antes, durante y después de la pandemia)
df['periodo'] = pd.cut(df['fecha_adj_firme'].dt.year,
                       bins=[2017, 2019, 2021, df['fecha_adj_firme'].dt.year.max()],
                       labels=[0, 1, 2])

# Filtrar el DataFrame para que solo queden los datos con las etiquetas especificadas
df = df[df['periodo'].notna()]
df.head()

,nro_sicop,nro_acto,fecha_adj_firme,cedula_institucion,fecha_publicacion,fecha_inicio_recepcion,fecha_cierre_recepcion,fecha_apertura,nro_procedimiento,tipo_procedimiento,modalidad_procedimiento,descripcion,total_procedimiento,duracion,periodo
0,20230200428,833534,2023-02-16 18:16:18,4000004017,2023-02-10 09:17:28.000,2023-02-10 09:20:00.000,2023-02-15 10:00:00,2023-02-15 10:00:00.000,2023PX-000033-0004900001,PROCEDIMIENTO POR EXCEPCIÓN,Cantidad definida,SE REQUIERE INSCRIBIR 2 FUNCIONARIOS DE LA DIV...,"218,280.00",1,2
1,20230300247,836311,2023-03-06 12:51:40,4000004017,2023-03-02 17:46:47.000,2023-03-02 17:48:00.000,2023-03-06 10:00:00,2023-03-06 10:00:00.000,2023PX-000054-0004900001,PROCEDIMIENTO POR EXCEPCIÓN,Servicios,Inscripción a 5 funcionarios de la Sugese:1. A...,"765,000.00",0,2
2,20230201566,835891,2023-03-03 08:34:17,4000004017,2023-02-27 16:09:16.000,2023-02-27 16:15:00.000,2023-03-02 10:00:00,2023-03-02 10:00:00.000,2023PX-000050-0004900001,PROCEDIMIENTO POR EXCEPCIÓN,Cantidad definida,SE REQUIERE INSCRIBIR 1 FUNCIONARIA DE LA DIVI...,"91,800.00",0,2
3,20230301392,840622,2023-03-27 13:32:29,4000004017,2023-03-17 10:06:09.000,2023-03-17 10:30:00.000,2023-03-24 10:30:00,2023-03-24 10:30:00.000,2023PX-000064-0004900001,PROCEDIMIENTO POR EXCEPCIÓN,Servicios,Inscripción curso: Análisis de datos en Micros...,"408,000.00",3,2
4,20230302152,841318,2023-03-28 22:00:32,4000004017,2023-03-24 08:12:06.000,2023-03-24 09:00:00.000,2023-03-28 09:00:00,2023-03-28 09:00:00.000,2023PX-000074-0004900001,PROCEDIMIENTO POR EXCEPCIÓN,Servicios,"Inscripción: El compromiso en el liderazgo, un...","6,426,875.00",0,2


In [19]:
df['fecha_cierre_recepcion'] = pd.to_datetime(df['fecha_cierre_recepcion'])
df['fecha_adj_firme'] = pd.to_datetime(df['fecha_adj_firme'])

# Calcular la diferencia en días y crear la nueva columna 'duracion'
df['duracion_proc'] = (df['fecha_adj_firme'] - df['fecha_cierre_recepcion']).dt.days


In [20]:
df_periodo_0 = df[df['periodo'] == 0]
df_periodo_1 = df[df['periodo'] == 1]
df_periodo_2 = df[df['periodo'] == 2]

#### Continúe con los gráficos...